In [4]:
from langgraph.prebuilt import create_react_agent
from langchain_community.chat_models import ChatTongyi
import os

# print(os.environ['DASHSCOPE_API_KEY'])
chat_model = ChatTongyi(model='qwen-max')


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

配置结构化输出

In [7]:
from pydantic import BaseModel
class WeatherResponse(BaseModel):
    conditions: str

In [9]:
# 添加记忆
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

agent = create_react_agent(
    model=chat_model,
    tools=[get_weather],
    checkpointer=checkpointer,
    response_format=WeatherResponse,
)

cnf = {"configurable": {"thread_id": "1"}}
sf_resp = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    cnf
)

ny_resp = agent.invoke(
    {"messages": [{"role": "user", "content": "what about new york?"}]},
    cnf
)

print(ny_resp)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='0e484bd6-3ea5-43fb-856b-a6bad7b56ec3'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'name': 'get_weather', 'arguments': '{"city": "sf"}'}, 'index': 0, 'id': 'call_7fe25a14578f424cbd4765', 'type': 'function'}]}, response_metadata={'model_name': 'qwen-max', 'finish_reason': 'tool_calls', 'request_id': '57f1299b-8f9d-9a3f-9793-09264531a9cd', 'token_usage': {'input_tokens': 166, 'output_tokens': 17, 'total_tokens': 183, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--c102d477-93f0-44cd-8837-e8a6659e475a-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'call_7fe25a14578f424cbd4765', 'type': 'tool_call'}]), ToolMessage(content="It's always sunny in sf!", name='get_weather', id='9a4e4da7-a877-4eff-88b5-69b8e518d1a1', tool_call_id='call_7fe25a14578f424cbd4765'), AIMessage(content="It's always sunny in San Francisco! However, pl